In [1]:
using DataFrames, Distributions, DataFramesMeta

In [2]:
function gen_data(; n = 500000, mu = [0, 2], sigma = 1, pZ1 = 0.8)
    data = DataFrame(
        Obs_ID = 1:n,
        Z = (rand(Uniform(0,1), n) .> pZ1) .+ 1
        )

    @transform!(data, :Y = rand(Normal(0, sigma), n) + mu[:Z])    
    @select!(data, :Obs_ID, :Y)
    @transform!(data, :p_Y_given_Z_1 = pdf.(Normal(mu[1], sigma), :Y))
    @transform!(data, :p_Y_given_Z_2 = pdf.(Normal(mu[2], sigma), :Y))

    return data
  end

gen_data (generic function with 1 method)

In [3]:
data = gen_data(n = 500000, pZ1 = 0.8);
first(data, 6)

,Obs_ID,Y,p_Y_given_Z_1,p_Y_given_Z_2
,Int64,Float64,Float64,Float64
1,1,0.420934,0.365119,0.114674
2,2,0.0157625,0.398893,0.0557132
3,3,-0.162978,0.393679,0.0384585
4,4,2.45073,0.0198021,0.360409
5,5,1.52869,0.124011,0.357006
6,6,1.39203,0.151403,0.331624


In [4]:
function fit_model!(
    data; 
    pi_hat_0 = 0.5, 
    tolerance = 0.0001,
    max_iterations = 1000,
    progress = DataFrame(
        iter = 1:(max_iterations+1), 
        pi_hat = Vector{Float64}(undef, max_iterations+1), 
        ll = Vector{Float64}(undef, max_iterations+1), 
        ll_diff = Vector{Float64}(undef, max_iterations+1)
        )
    )

    pi_hat = pi_hat_0
    E_step!(data, pi_hat)
    ll = loglik(data)
    progress[1,:] = (0, pi_hat, ll, NaN)
    
    last_iter = 0
    for i in 1:max_iterations
        pi_hat = M_step(data)
        E_step!(data, pi_hat)
        
        ll_old = ll
        ll = loglik(data)
        ll_diff = ll - ll_old
        progress[i+1,:] = (i, pi_hat, ll, ll_diff)

        if ll_diff < tolerance
            last_iter = i
            break
        end
    end
    return progress[1:(last_iter + 1), :]
end

fit_model! (generic function with 1 method)

In [5]:
function E_step!(data, pi_hat)
    @transform!(data, :pY_Z1 = :p_Y_given_Z_1 .* pi_hat)
    @transform!(data, :pY_Z2 = :p_Y_given_Z_2 .* (1- pi_hat))
    @transform!(data, :pY = :pY_Z1 + :pY_Z2)
    @transform!(data, :pZ1_given_Y = :pY_Z1 ./ :pY)
end

E_step! (generic function with 1 method)

In [6]:
function M_step(data)
    mean(data[!, :pZ1_given_Y])
end

M_step (generic function with 1 method)

In [7]:
function loglik(data)
    sum(log.(data[!, :pY]))
end

loglik (generic function with 1 method)

In [8]:
@time progress = fit_model!(data, tolerance = .00001);


  0.814944 seconds (3.18 M allocations: 533.181 MiB, 7.68% gc time, 82.18% compilation time)


The first run requires compilation; on subsequent calls it will be faster:

In [9]:
@time progress = fit_model!(data, tolerance = .00001)

  0.128062 seconds (12.99 k allocations: 363.058 MiB, 14.40% gc time)


,iter,pi_hat,ll,ll_diff
,Int64,Float64,Float64,Float64
1,0,0.5,-8.77633e5,NaN
2,1,0.66514,-8.37182e5,40451.0
3,2,0.73724,-8.2827e5,8911.09
4,3,0.769564,-8.26214e5,2056.72
5,4,0.784832,-8.25715e5,499.179
6,5,0.792298,-8.2559e5,125.017
7,6,0.796022,-8.25558e5,31.857
8,7,0.797899,-8.2555e5,8.1924
9,8,0.798849,-8.25547e5,2.11678
